In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class Neural_Network:
    # Initialize the network
    def __init__(self, num_inputs, num_hidden, num_outputs, hidden_layer_weights, output_layer_weights, learning_rate):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.hidden_layer_weights = hidden_layer_weights
        self.output_layer_weights = output_layer_weights
        self.learning_rate = learning_rate

    def sigmoid(self, inputs):
        
        output = 1/(1 + np.exp(-inputs))
        return output

    def forward_pass(self, inputs):
        hidden_layer_outputs = []
        hidden_biases = [-0.02,-0.2]
        for i in range(self.num_hidden): #2
            weighted_sum = 0.
            for j in range(len(self.hidden_layer_weights)): #4
                weighted_sum += inputs[j] * self.hidden_layer_weights[j][i]
            output = self.sigmoid(weighted_sum)
            hidden_layer_outputs.append(output)

        output_layer_outputs = []
        output_biases = [-0.33,0.26,0.06]
        for i in range(self.num_outputs): #3
            weighted_sum = 0.
            for j in range(len(self.output_layer_weights)):#2
                weighted_sum += hidden_layer_outputs[j] * self.output_layer_weights[j][i]
            output = self.sigmoid(weighted_sum)
            output_layer_outputs.append(output)
        return hidden_layer_outputs, output_layer_outputs

    def backward_propagate_error(self, inputs, hidden_layer_outputs, output_layer_outputs, desired_outputs):

        output_layer_betas = np.zeros(self.num_outputs)
        for i in range(self.num_outputs):
            output_layer_betas[i] = desired_outputs[i] - output_layer_outputs[i]

        hidden_layer_betas = np.zeros(self.num_hidden)
        for i in range (self.num_hidden): 
            for j in range(self.num_outputs):
                hidden_layer_betas[i] += self.output_layer_weights[i][j] * output_layer_outputs[j] * (1 - output_layer_outputs[j]) * output_layer_betas[j] 

        delta_output_layer_weights = np.zeros((self.num_hidden, self.num_outputs))
        for i in range(self.num_hidden):
            for j in range(self.num_outputs):
                delta_output_layer_weights[i][j] = self.learning_rate * hidden_layer_outputs[i] * output_layer_outputs[j] * (1 - output_layer_outputs[j] ) * output_layer_betas[j] 
        
        delta_hidden_layer_weights = np.zeros((self.num_inputs, self.num_hidden))
        for i in range(self.num_inputs):
            for j in range(self.num_hidden):
                delta_hidden_layer_weights[i][j] = self.learning_rate * inputs[i] * hidden_layer_outputs[j] * ( 1 - hidden_layer_outputs[j]) * hidden_layer_betas[j] 
       
        return delta_output_layer_weights, delta_hidden_layer_weights

    def update_weights(self, delta_output_layer_weights, delta_hidden_layer_weights):
        self.hidden_layer_weights += delta_hidden_layer_weights
        self.output_layer_weights += delta_output_layer_weights

    def train(self, instances, desired_outputs, epochs):

        for epoch in range(epochs):
            predictions = []
            for i, instance in enumerate(instances):
                hidden_layer_outputs, output_layer_outputs = self.forward_pass(instance)
                delta_output_layer_weights, delta_hidden_layer_weights, = self.backward_propagate_error(
                    instance, hidden_layer_outputs, output_layer_outputs, desired_outputs[i])
                predicted_class = np.argmax(output_layer_outputs)   # TODO!
                predictions.append(predicted_class)

                self.update_weights(delta_output_layer_weights, delta_hidden_layer_weights)

    def predict(self, instances):
        predictions = []
        for instance in instances:
            hidden_layer_outputs, output_layer_outputs = self.forward_pass(instance)
            predicted_class = np.argmax(output_layer_outputs)  # TODO! Should be 0, 1, or 2.
            predictions.append(predicted_class)
        return predictions

In [8]:
def encode_labels(labels):
    # encode 'Adelie' as 1, 'Chinstrap' as 2, 'Gentoo' as 3
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    # don't worry about this
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

    # encode 1 as [1, 0, 0], 2 as [0, 1, 0], and 3 as [0, 0, 1] (to fit with our network outputs!)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    return label_encoder, integer_encoded, onehot_encoder, onehot_encoded


def test_learning_rate(lr):
    data = pd.read_csv('penguins307-train.csv')
    labels = data.iloc[:, -1]
    instances = data.iloc[:, :-1]
    scaler = MinMaxScaler()
    instances = scaler.fit_transform(instances)
    label_encoder, integer_encoded, onehot_encoder, onehot_encoded = encode_labels(labels)

    n_in = 4
    n_hidden = 2
    n_out = 3
    learning_rate = lr

    initial_hidden_layer_weights = np.array([[-0.28, -0.22], [0.08, 0.20], [-0.30, 0.32], [0.10, 0.01]])
    initial_output_layer_weights = np.array([[-0.29, 0.03, 0.21], [0.08, 0.13, -0.36]])

    nn = Neural_Network(n_in, n_hidden, n_out, initial_hidden_layer_weights, initial_output_layer_weights,
                        learning_rate)


    nn.train([instances[0]], [onehot_encoded[0]], 1)

    nn.train(instances,onehot_encoded , 100)

    pd_data_ts = pd.read_csv('penguins307-test.csv')
    test_labels = pd_data_ts.iloc[:, -1]
    test_instances = pd_data_ts.iloc[:, :-1]
    #scale the test according to our training data.
    test_instances = scaler.transform(test_instances)
    pred = nn.predict(test_instances)
    test_label_encoder, test_integer_encoded, test_onehot_encoder, test_onehot_encoded = encode_labels(test_labels)
    test_labels = test_onehot_encoded
    acc = 0
    for i in range(len(pred)):
        if pred[i] == np.argmax(test_labels[i]):
            acc+=1
    print('Accuracy in test set: = ', acc/len(test_labels))
    return acc


In [16]:
test_learning_rate(10)
test_learning_rate(0.2)
test_learning_rate(0.001)
test_learning_rate(0.0001)
test_learning_rate(0.0000001)

Accuracy in test set: =  0.6461538461538462
Accuracy in test set: =  0.8
Accuracy in test set: =  0.4461538461538462
Accuracy in test set: =  0.2
Accuracy in test set: =  0.2


13

In [17]:
def test_epochs(epoch):
    data = pd.read_csv('penguins307-train.csv')
    labels = data.iloc[:, -1]
    instances = data.iloc[:, :-1]
    scaler = MinMaxScaler()
    instances = scaler.fit_transform(instances)
    label_encoder, integer_encoded, onehot_encoder, onehot_encoded = encode_labels(labels)

    n_in = 4
    n_hidden = 2
    n_out = 3
    learning_rate = 0.2

    initial_hidden_layer_weights = np.array([[-0.28, -0.22], [0.08, 0.20], [-0.30, 0.32], [0.10, 0.01]])
    initial_output_layer_weights = np.array([[-0.29, 0.03, 0.21], [0.08, 0.13, -0.36]])

    nn = Neural_Network(n_in, n_hidden, n_out, initial_hidden_layer_weights, initial_output_layer_weights,
                        learning_rate)


    nn.train([instances[0]], [onehot_encoded[0]], 1)

    nn.train(instances,onehot_encoded , epoch)

    pd_data_ts = pd.read_csv('penguins307-test.csv')
    test_labels = pd_data_ts.iloc[:, -1]
    test_instances = pd_data_ts.iloc[:, :-1]
    #scale the test according to our training data.
    test_instances = scaler.transform(test_instances)
    pred = nn.predict(test_instances)
    test_label_encoder, test_integer_encoded, test_onehot_encoder, test_onehot_encoded = encode_labels(test_labels)
    test_labels = test_onehot_encoded
    acc = 0
    for i in range(len(pred)):
        if pred[i] == np.argmax(test_labels[i]):
            acc+=1
    print('Accuracy in test set: = ', acc/len(test_labels))
    return acc


In [20]:
test_epochs(1)
test_epochs(10)
test_epochs(100)
test_epochs(500)
test_epochs(10000)

Accuracy in test set: =  0.35384615384615387
Accuracy in test set: =  0.8
Accuracy in test set: =  0.8153846153846154
Accuracy in test set: =  0.8923076923076924
Accuracy in test set: =  0.9076923076923077


59